In [1]:
###LIBRARIES
%matplotlib notebook
%config InlineBackend.figure_format = 'retina'
import cv2
import numpy as np
from matplotlib import pyplot as plt
import shutil
from pathlib2 import Path
import pandas as pd
import sys
from enum import Enum
import os
import time

## Global variables -- Path to database files

In [32]:
#GLOBAL VARIABLES
#PAth to folder with database
str_db_dir = '../../../data/CADDY_gestures_complete_v2_release/'
str_csvDatabase = 'CADDY_gestures_all_true_positives_release_v2.csv'
extract_ROI = False
str_ROI_dir = 'hands-ROIs/'
shutil_ignore_patterns = ('*.jpg','*~','*.csv*')

### Create directory tree for hand ROIs

#### Do this only once

In [33]:
#SNIPPETS
#copy a directory tree with or without files
def copytree(src, dst, symlinks=False, ignore=None, only_dirs=True):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s) and item != str_ROI_dir[:-1]:
            shutil.copytree(s, d, symlinks, ignore)
        elif not only_dirs:
            shutil.copy2(s, d)

#go through each scenario and create the directory tree for the haar cascade database
#following the same dir tree layout as in the database (folders of synthetic and non-synthetic images)
if os.path.isdir(str_db_dir+str_ROI_dir) is False:
    os.mkdir(str_db_dir+str_ROI_dir)
dst = os.path.join(str_db_dir,str_ROI_dir)
copytree(str_db_dir, dst, False, ignore=shutil.ignore_patterns(*shutil_ignore_patterns),only_dirs=True)

### Read database -- only raw data

In [43]:
#Read original file with non-altered stereo images
tmp_df = pd.read_csv(str_db_dir+str_csvDatabase)
database_source_df = tmp_df.loc[tmp_df['synthetic'] == 0]
database_source_df.head(10)

,index,scenario,stereo left,stereo right,label name,label id,roi left,roi right,synthetic,iqa_mdm_entropy,iqa_mdm_d,iqa_mdm_dcomp,distortion,param 1,param 2
0,0,biograd-A,/biograd-A/true_positives/raw/biograd-A_00000_...,/biograd-A/true_positives/raw/biograd-A_00000_...,num_delimiter,10,"[237,236,54,65]","[155,236,54,65]",0,6.971026,0.957653,0.902000,NaN,NaN,NaN
10,10,biograd-A,/biograd-A/true_positives/raw/biograd-A_00001_...,/biograd-A/true_positives/raw/biograd-A_00001_...,num_delimiter,10,"[230,230,61,79]","[147,230,61,79]",0,6.977803,0.958442,0.885512,NaN,NaN,NaN
20,20,biograd-A,/biograd-A/true_positives/raw/biograd-A_00002_...,/biograd-A/true_positives/raw/biograd-A_00002_...,num_delimiter,10,"[222,230,72,75]","[139,230,72,75]",0,6.930135,0.958565,0.885733,NaN,NaN,NaN
30,30,biograd-A,/biograd-A/true_positives/raw/biograd-A_00003_...,/biograd-A/true_positives/raw/biograd-A_00003_...,num_delimiter,10,"[231,231,62,83]","[148,231,62,83]",0,6.948746,0.958243,0.885593,NaN,NaN,NaN
40,40,biograd-A,/biograd-A/true_positives/raw/biograd-A_00004_...,/biograd-A/true_positives/raw/biograd-A_00004_...,num_delimiter,10,"[222,219,70,76]","[138,219,70,76]",0,7.175039,0.955913,0.875217,NaN,NaN,NaN
50,50,biograd-A,/biograd-A/true_positives/raw/biograd-A_00005_...,/biograd-A/true_positives/raw/biograd-A_00005_...,num_delimiter,10,"[223,220,68,81]","[139,220,68,81]",0,7.164511,0.956243,0.894149,NaN,NaN,NaN
60,60,biograd-A,/biograd-A/true_positives/raw/biograd-A_00006_...,/biograd-A/true_positives/raw/biograd-A_00006_...,five,15,"[220,209,86,90]","[136,209,86,90]",0,7.118584,0.957484,0.917649,NaN,NaN,NaN
70,70,biograd-A,/biograd-A/true_positives/raw/biograd-A_00007_...,/biograd-A/true_positives/raw/biograd-A_00007_...,five,15,"[210,200,110,118]","[127,200,110,118]",0,7.057821,0.957349,0.895427,NaN,NaN,NaN
80,80,biograd-A,/biograd-A/true_positives/raw/biograd-A_00008_...,/biograd-A/true_positives/raw/biograd-A_00008_...,five,15,"[229,205,98,101]","[145,205,98,101]",0,6.981269,0.957604,0.893355,NaN,NaN,NaN
90,90,biograd-A,/biograd-A/true_positives/raw/biograd-A_00009_...,/biograd-A/true_positives/raw/biograd-A_00009_...,five,15,"[233,199,98,115]","[150,199,98,115]",0,6.957809,0.958578,0.900619,NaN,NaN,NaN


In [51]:
#Iterate through database
file_cc = 0
stereo_view = ['left', 'right']
for index, row in database_source_df.iterrows():
    if file_cc > -1: #hackable condition helpful if few loops want to be tested
        for view in stereo_view:
            #select left or right stereo pair        
            str_selected_img = str_db_dir + row['stereo '+view][1::]
            #str_selected_img = str_db_dir + '/biograd-C/true_positives/raw/biograd-C_00080_right.jpg'
            #read image
            img = cv2.imread(str_selected_img,cv2.IMREAD_COLOR)
        
            #extract roi info from csv file
            if row['roi '+view]:
                try: #avoid empty cells
                    str_roi_params = row['roi '+view][1:-1] #get rid of the brackets
                    str_roi_params = str_roi_params.replace(";",",") #in case of more than one ROI
                    roi_params = str_roi_params.split(",")
                    no_rois = int(len(roi_params) / 4) #check number of ROIs
                    #print(roi_params)

                    #crop each ROI in the image
                    for i in range(0,no_rois):
                        #get params of the ROI
                        topx = int(roi_params[4*i])
                        topy = int(roi_params[(4*i)+1])
                        width = int(roi_params[(4*i)+2])
                        height = int(roi_params[(4*i)+3])
                        #extract ROI
                        #print(topx, topy, width, height)
                        patch_image = img[topy:(topy+height-1), topx:(topx+width-1)]
                        #set file path to save the ROI
                        str_patch_path = (str_db_dir + str_ROI_dir + row['stereo '+view][1::])
                        #print(str_patch_path)
                        cv2.imwrite(str_patch_path,patch_image)
                except:
                    print(str_selected_img)
        
            #See progress
            time.sleep(.1)
            file_cc+=1
            sys.stdout.write("\rProgress: " + str(file_cc) + " images ...")

Progress: 5565 images ...../../../data/CADDY_gestures_complete_v2_release/biograd-C/true_positives/raw/biograd-C_00079_right.jpg
Progress: 5593 images ...../../../data/CADDY_gestures_complete_v2_release/biograd-C/true_positives/raw/biograd-C_00093_right.jpg
Progress: 5597 images ...../../../data/CADDY_gestures_complete_v2_release/biograd-C/true_positives/raw/biograd-C_00095_right.jpg
Progress: 5727 images ...../../../data/CADDY_gestures_complete_v2_release/biograd-C/true_positives/raw/biograd-C_00160_right.jpg
Progress: 5826 images ...../../../data/CADDY_gestures_complete_v2_release/biograd-C/true_positives/raw/biograd-C_00210_left.jpg
Progress: 5828 images ...../../../data/CADDY_gestures_complete_v2_release/biograd-C/true_positives/raw/biograd-C_00211_left.jpg
Progress: 5830 images ...../../../data/CADDY_gestures_complete_v2_release/biograd-C/true_positives/raw/biograd-C_00212_left.jpg
Progress: 5907 images ...../../../data/CADDY_gestures_complete_v2_release/biograd-C/true_positives/r

Progress: 12347 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/genova-A_00254_right.jpg
Progress: 12385 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/genova-A_00273_right.jpg
Progress: 12387 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/genova-A_00274_right.jpg
Progress: 12388 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/genova-A_00275_left.jpg
Progress: 12471 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/genova-A_00316_right.jpg
Progress: 12473 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/genova-A_00317_right.jpg
Progress: 12475 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/genova-A_00318_right.jpg
Progress: 12477 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/ge

Progress: 16045 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/genova-A_02103_right.jpg
Progress: 16049 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/genova-A_02105_right.jpg
Progress: 16475 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/genova-A_02318_right.jpg
Progress: 16477 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/genova-A_02319_right.jpg
Progress: 16479 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/genova-A_02320_right.jpg
Progress: 16546 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/genova-A_02354_left.jpg
Progress: 16574 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/genova-A_02368_left.jpg
Progress: 16576 images ...../../../data/CADDY_gestures_complete_v2_release/genova-A/true_positives/raw/gen